Rebecca Black • March 30, 2016

This notebook contains code written for my analysis of New York Subway ridership data.
I performed a variety of analyses on these data, which included incorporating weather
data and doing a linear regression analysis using gradient descent. This analysis is
presented in "unexecuted" form, as a code sample.

First, I use pandasql to select out and add up the total # of rainy days in the weather dataset:

In [ ]:
import pandas
import pandasql

weather_data = pandas.read_csv(weather_underground.csv)

q =
SELECT sum(rain)
FROM weather_data
WHERE cast(rain as integer) = 1
    
#Execute the SQL command against the DataFrame
rainy_days = pandasql.sqldf(q.lower(), locals())
print rainy_days

Next I use pandasql to subset the data according to the boolean 'fog', and return the
maximum max temperature for the foggy and non-foggy days in the dataset:

In [ ]:
weather_data = pandas.read_csv(weather_underground.csv)

q =
SELECT fog, max(maxtempi)
FROM weather_data
GROUP BY fog

#Execute the SQL command against the DataFrame
foggy_days = pandasql.sqldf(q.lower(), locals())
print foggy_days

Now I look at the weekends. The aim here was to determine the average mean temperature
across all the weekend days in the dataset. It was necessary to convert the dates to
days of the week to restrict the selection to weekends:

In [ ]:
weather_data = pandas.read_csv(weather_underground.csv)

q =
SELECT avg(meantempi)
FROM weather_data
WHERE cast(strftime('%w', date) as integer)=0 OR cast(strftime('%w', date) as integer)=6

#Execute the SQL command against the DataFrame
mean_temp_weekends = pandasql.sqldf(q.lower(), locals())
print mean_temp_weekends

Now I look at a more complex query. Here I look at the average minimum temperature
on rainy days where the minimum temperature for that day was more than 55 degrees:

In [ ]:
weather_data = pandas.read_csv(weather_underground.csv)

q =
SELECT avg(mintempi)
FROM weather_data
WHERE cast(rain as integer) = 1 AND cast(mintempi as integer) > 55
GROUP BY rain
    
#Execute the SQL command against the DataFrame
avg_min_temp_rainy = pandasql.sqldf(q.lower(), locals())
print avg_min_temp_rainy


Next, I turn to the actual subway turnstile data. Before proceeding with any
analysis, it is necessary to rewrite the rows of the dataset so that we have 
only one datapoint per row (the raw dataset has five datapoints per row.) 
This requires splitting each line appropriately, taking into account that the 
first three elements of each line in the raw dataset should be repeated for 
each observation:

In [ ]:
import csv

for name in filenames:
    f_in = open(name, 'r')
    f_out = open("updated_" + name, 'w')

    reader = csv.reader(f_in, delimiter=',')
    writer = csv.writer(f_out, delimiter=',')

    for line in reader:
        elements = len(line)
        num_lines = (elements - 3) / 5
        for i in range(num_lines):
            line_out = [line[0], line[1], line[2], line[3+5*i], line[4+5*i], line[5+5*i], line[6+5*i], line[7+5*i]]
            writer.writerow(line_out)

    f_in.close()
    f_out.close()

Now I write a function that combines a group of files into a single file
to facilitate easier modeling:

In [ ]:
def create_master_turnstile_file(filenames, output_file):
    with open(output_file, 'w') as master_file:
        master_file.write('C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn\n')
        for filename in filenames:
            with open(filename) as infile:
                for line in infile:
                    master_file.write(line)

Here I'm subsetting the data according to a specific attribute of DESCn:

In [ ]:
filtered = pandas.read_csv(filename)
filtered_df = pandas.DataFrame(filename)

q=
SELECT * 
FROM filtered_df 
WHERE DESCn like 'REGULAR'

turnstile_data = pandasql.sqldf(q.lower(), locals())
return turnstile_data

Now I use the cumulative entries and exits to produce a calculated variable
that gives us the number of entries per hour:

In [ ]:
df['ENTRIESn_hourly'] = (df['ENTRIESn'] - df['ENTRIESn'].shift(1)).fillna(1)

Now I do the same to produce a calculated variable that gives us the number of
exits per hour:

In [ ]:
df['EXITSn_hourly'] = (df['EXITSn'] - df['EXITSn'].shift(1)).fillna(0)

Now I write a function to extract the hour portion from the time
variable:

In [ ]:
def time_to_hour(time):
    hour=time[0:time.find(':')]
    hour=int(hour)
    return hour


Now I reformat the dates in the subway dataset so they match the format
from the weather dataset:

In [ ]:
from datetime import datetime

def reformat_subway_dates(date):
    t = datetime.strptime(date, '%m-%d-%y')
    date_formatted = t.strftime('%Y-%m-%d')
    return date_formatted

Next is a histogram of the hourly entries grouped according to rain:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

turnstile_weather_rainy=turnstile_weather[turnstile_weather["rain"] == 1]
turnstile_weather_norain=turnstile_weather[turnstile_weather["rain"] == 0]
    
plt.figure()
turnstile_weather_rainy['ENTRIESn_hourly'].hist()
turnstile_weather_norain['ENTRIESn_hourly'].hist()
return plt

Here I perform a Mann-Whitney U test to determine whether subway ridership
is the same on rainy days as it is on non-rainy days:

In [ ]:
import scipy
import scipy.stats

turnstile_weather_rainy=turnstile_weather[turnstile_weather["rain"] == 1]
turnstile_weather_norain=turnstile_weather[turnstile_weather["rain"] == 0]

with_rain_mean=np.mean(turnstile_weather_rainy['ENTRIESn_hourly'])
without_rain_mean=np.mean(turnstile_weather_norain['ENTRIESn_hourly'])

#Perform a Mann-Whitney U test (null hypothesis: two populations are from same distribution)
#U is the test statistic, p is the one sided p-value for the test
U,p=scipy.stats.mannwhitneyu(turnstile_weather_rainy['ENTRIESn_hourly'],turnstile_weather_norain['ENTRIESn_hourly'])
    
print with_rain_mean, without_rain_mean, U, p

I've omitted the code for the regression, but the next step is making a histogram
of the residuals for the regression to check that they do not exhibit a pattern:

In [ ]:
plt.figure()
(turnstile_weather['ENTRIESn_hourly'] - predictions).hist()
return plt

Finally, here is a function that computes the R^2 of the residuals from the regression:

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import sys

def compute_r_squared(data, predictions):
    difference_data_pred=data-predictions
    difference_data_avg=data-np.mean(data)
    r_squared=1-(np.dot(difference_data_pred,difference_data_pred)/np.dot(difference_data_avg,difference_data_avg))
    
    return r_squared